In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

In [12]:
import torch
import torch.nn as nn

# XOR 데이터셋 정의
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# MSE
def mse_loss(predictions, targets):
    return torch.mean((predictions - targets) ** 2)

# tanh
def tanh(x):
    return (torch.exp(x) - torch.exp(-x)) / (torch.exp(x) + torch.exp(-x))

# 1-layer MLP 모델 정의
class MLP1(nn.Module):
    def __init__(self):
        super(MLP1, self).__init__()
        self.fc1 = nn.Linear(2, 1)
        #self.tanh = nn.Tanh()
    
    def forward(self, x):
        x = self.fc1(x)
        x = tanh(x)
        return x

# 2-layer MLP 모델 정의
class MLP2(nn.Module):
    def __init__(self):
        super(MLP2, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)
        #self.tanh = nn.Tanh()
    
    def forward(self, x):
        x = self.fc1(x)
        x = tanh(x)
        x = self.fc2(x)
        x = tanh(x)
        return x

# 모델 인스턴스 생성
model1 = MLP1()
model2 = MLP2()

# 1-layer MLP 모델 학습
epochs = 1000

optimizer1 = torch.optim.SGD(model1.parameters(), lr=0.1)
for epoch in range(epochs):
    optimizer1.zero_grad()
    outputs1 = model1(X)
    loss1 = mse_loss(outputs1, y)
    loss1.backward()
    optimizer1.step()

# 2-layer MLP 모델 학습
optimizer2 = torch.optim.SGD(model2.parameters(), lr=0.1)
for epoch in range(epochs):
    optimizer2.zero_grad()
    outputs2 = model2(X)
    loss2 = mse_loss(outputs2, y)
    loss2.backward()
    optimizer2.step()

# 테스트 데이터 예측
test_outputs1 = model1(X)
test_outputs2 = model2(X)
predictions1 = (test_outputs1 > 0.5).float()
predictions2 = (test_outputs2 > 0.5).float()

# 결과 출력
print("1-layer MLP Predictions:", predictions1)
print("2-layer MLP Predictions:", predictions2)


1-layer MLP Predictions: tensor([[0.],
        [0.],
        [0.],
        [1.]])
2-layer MLP Predictions: tensor([[0.],
        [1.],
        [1.],
        [0.]])


In [5]:
# 데이터 전처리를 위한 변환 선언
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 데이터셋 불러오기
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(64 * 4 * 4, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [8]:
# 하이퍼파라미터 설정
batch_size = 64
lr = 0.0001
epochs = 10

# 데이터 로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 생성 및 손실 함수, 최적화 알고리즘 설정
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# 모델 학습
for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Test Accuracy: {accuracy:.2f}%")


Epoch 1/25, Test Accuracy: 40.32%
Epoch 2/25, Test Accuracy: 45.86%
Epoch 3/25, Test Accuracy: 48.09%
Epoch 4/25, Test Accuracy: 50.43%
Epoch 5/25, Test Accuracy: 52.25%
Epoch 6/25, Test Accuracy: 53.47%
Epoch 7/25, Test Accuracy: 54.53%
Epoch 8/25, Test Accuracy: 55.32%
Epoch 9/25, Test Accuracy: 56.11%
Epoch 10/25, Test Accuracy: 56.38%
Epoch 11/25, Test Accuracy: 57.51%
Epoch 12/25, Test Accuracy: 58.33%
Epoch 13/25, Test Accuracy: 59.40%
Epoch 14/25, Test Accuracy: 60.07%
Epoch 15/25, Test Accuracy: 60.58%
Epoch 16/25, Test Accuracy: 61.68%
Epoch 17/25, Test Accuracy: 61.81%
Epoch 18/25, Test Accuracy: 62.12%
Epoch 19/25, Test Accuracy: 62.61%
Epoch 20/25, Test Accuracy: 62.99%
Epoch 21/25, Test Accuracy: 63.55%
Epoch 22/25, Test Accuracy: 63.69%
Epoch 23/25, Test Accuracy: 64.39%
Epoch 24/25, Test Accuracy: 64.68%
Epoch 25/25, Test Accuracy: 64.14%
